<a href="https://colab.research.google.com/github/Murthy-Kolluru/modelthinking/blob/main/Session06Prompting-Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount google drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#Create folder
import os

# Define the directory path
directory_path = '/content/drive/My Drive/stocks'

# Create the directory if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)


In [ ]:
#Download data

import yfinance as yf
import pandas as pd
from datetime import datetime

# List of stock tickers
stock_tickers = ['AAPL', 'MSFT', 'F', 'CSCO', 'PFE', 'WMT', 'JPM', 'ACN', 'JNJ', 'PG']

# Fetch historical data for each stock
stock_data = {}
for ticker in stock_tickers:
    stock_data[ticker] = yf.download(ticker, start="2004-01-01", end="2024-01-01")

# Extract daily closing values
closing_values = {}
for ticker, data in stock_data.items():
    closing_values[ticker] = data['Close']

# Write closing values to CSV and store in Google Drive
for ticker, closes in closing_values.items():
    file_name = f"{ticker}_closing_values.csv"
    closes.to_csv(f"/content/drive/My Drive/stocks/{file_name}")

# Display first five and last five values in table format
#print("First five and last five values of all stocks:")
#for ticker, closes in closing_values.items():
 #   print(f"\n{ticker}")
  #  print(pd.concat([closes.head(), closes.tail()]))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
#Create annual data files for each stock

def split_into_annual_files(stock_data, directory_path):
    # Iterate over each stock's data
    for ticker, data in stock_data.items():
        print(f"Processing data for {ticker}...")

        # Create folder for the stock if it doesn't exist
        stock_folder = os.path.join(directory_path, ticker)
        if not os.path.exists(stock_folder):
            os.makedirs(stock_folder)

        # Split data into annual stock values and save as separate files
        for year in range(data.index.year.min(), data.index.year.max() + 1):
            annual_data = data[str(year)]
            file_name = f"{year}_closing_values.csv"
            file_path = os.path.join(stock_folder, file_name)
            annual_data.to_csv(file_path)

# Example usage
split_into_annual_files(stock_data, '/content/drive/My Drive/stocks')


Processing data for AAPL...


<ipython-input-33-b7da15fdf3c1>:15: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  annual_data = data[str(year)]


Processing data for MSFT...
Processing data for F...
Processing data for CSCO...
Processing data for PFE...
Processing data for WMT...
Processing data for JPM...
Processing data for ACN...
Processing data for JNJ...
Processing data for PG...


In [ ]:
#Delete the original 20 year files

def delete_original_files(directory_path):
    # Iterate over files in the directory
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            # Check if the file is a CSV file containing the original 20-year data
            if file.endswith("_closing_values.csv") and not any(f"{year}_" in file for year in range(2004, 2024)):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted {file_path}")

# Example usage
delete_original_files('/content/drive/My Drive/stocks')



Deleted /content/drive/My Drive/stocks/AAPL_closing_values.csv
Deleted /content/drive/My Drive/stocks/MSFT_closing_values.csv
Deleted /content/drive/My Drive/stocks/F_closing_values.csv
Deleted /content/drive/My Drive/stocks/CSCO_closing_values.csv
Deleted /content/drive/My Drive/stocks/PFE_closing_values.csv
Deleted /content/drive/My Drive/stocks/WMT_closing_values.csv
Deleted /content/drive/My Drive/stocks/JPM_closing_values.csv
Deleted /content/drive/My Drive/stocks/ACN_closing_values.csv
Deleted /content/drive/My Drive/stocks/JNJ_closing_values.csv
Deleted /content/drive/My Drive/stocks/PG_closing_values.csv


In [ ]:
#Compute and store annual returns and risks
import pandas as pd
import os
import numpy as np

def compute_annual_return_and_risk(stock_data, directory_path):
    # Create an empty DataFrame to store annual returns and risks for each ticker and year
    annual_return_risk_df = pd.DataFrame()

    # Create an empty list to store years
    years = []

    # Iterate through every ticker and every year
    for ticker, data in stock_data.items():
        print(f"Processing data for {ticker}...")

        # Create empty lists to store annual returns and risks for each year
        annual_returns = []
        annual_risks = []

        for year in range(data.index.year.min(), data.index.year.max() + 1):
            annual_data = data.loc[str(year)]

            # Compute daily returns starting from the second data point in each year
            daily_returns = annual_data['Close'].pct_change().dropna()

            # Calculate annual return for the year
            annual_return = daily_returns.mean()

            # Calculate annual risk (standard deviation) for the year
            annual_risk = daily_returns.std()

            # Append year to the list if it's not already in the list
            if year not in years:
                years.append(year)

            # Append annual return and risk to the respective lists
            annual_returns.append(annual_return)
            annual_risks.append(annual_risk)

        # Pad the lists with NaN values for the missing years
        annual_returns.extend([np.nan] * (len(annual_return_risk_df) - len(annual_returns)))
        annual_risks.extend([np.nan] * (len(annual_return_risk_df) - len(annual_risks)))

        # Add the 'Year' column to the DataFrame
        annual_return_risk_df['Year'] = years

        # Add annual returns and risks to the DataFrame
        annual_return_risk_df[f"{ticker}-returns"] = annual_returns
        annual_return_risk_df[f"{ticker}-risks"] = annual_risks

    # Store the DataFrame in a CSV file
    annual_return_risk_df.to_csv(os.path.join(directory_path, "annual-return-risk.csv"))

# Example usage
compute_annual_return_and_risk(stock_data, '/content/drive/My Drive/stocks')



Processing data for AAPL...
Processing data for MSFT...
Processing data for F...
Processing data for CSCO...
Processing data for PFE...
Processing data for WMT...
Processing data for JPM...
Processing data for ACN...
Processing data for JNJ...
Processing data for PG...
